In [1]:
def read_maps(size, n_sp, n_tar, id,n_rob):
    map_start = []
    map_end = []
    T = 0
    robots_start = []
    
    def read_file(filename):
        with open(filename, "r") as file:
            T = int(file.readline())
            num_robots = int(file.readline())
            robots = []
            for _ in range(num_robots):
                x, y = map(int, file.readline().strip().split(','))
                robots.append((x, y))
            
            map_data = []
            for line in file:
                map_data.append(list(map(int, line.strip().split(','))))
                
        return T, robots, map_data

    # Read start file
    start_filename = f"data_graphform\\{size[0]}x{size[1]}\\tar{n_tar}\\start\\graph_start_{size[0]}x{size[1]}_tar{n_tar}_sp{n_sp}_rob{n_rob}_{id}.txt"
    T, robots, map_start = read_file(start_filename)
    
    # Read end file
    end_filename = f"data_graphform\\{size[0]}x{size[1]}\\tar{n_tar}\\end\\graph_end_{size[0]}x{size[1]}_tar{n_tar}_sp{n_sp}_rob{n_rob}_{id}.txt"
    _, _, map_end = read_file(end_filename)
    
    return T, map_start, map_end, robots

def set_parameter(size, n_sp, n_tar, id):
    grid_size = size[0]
    num_cargos = size[0] * size[1] - n_sp
    alpha = 0.001
    return grid_size, num_cargos, alpha

def set_node(size, map_start, map_end, robots):
    cargo_init_pos = []
    cargo_goal_pos = []
    non_target = []
    
    for i in range(size[0]):
        for j in range(size[1]):
            if map_start[i][j] > 0:
                cargo_init_pos.append((map_start[i][j]-1, i, j))
            elif map_start[i][j] < 0:
                non_target.append((-1, i, j))
    
    for non in non_target:
        cargo_init_pos.append((non[0]+1+len(cargo_init_pos), non[1], non[2]))
    
    cargo_init_pos = sorted(cargo_init_pos, key=lambda x:x[0])
    
    cargo_goal_pos = [(map_end[i][j]-1, i, j) for i in range(size[0]) 
                      for j in range(size[1]) if map_end[i][j] > 0]
    cargo_goal_pos = sorted(cargo_goal_pos, key=lambda x:x[0])
    
    cargos, targets = len(cargo_init_pos), len(cargo_goal_pos)
    for _ in range(cargos - targets):
        cargo_goal_pos.append((None, None, None))
    
    initial_positions = [(i, x, y) for i, (x, y) in enumerate(robots)]
    
    return initial_positions, cargo_init_pos, cargo_goal_pos

def preprocess(size, n_sp, n_tar, id,n_rob):
    T, map_start, map_end, robots = read_maps(size, n_sp, n_tar, id,n_rob)
    grid_size, num_cargos, alpha = set_parameter(size, n_sp, n_tar, id)
    initial_positions, cargo_init_pos, cargo_goal_pos = set_node(size, map_start, map_end, robots)
    num_robots = len(robots)
    
    return (T, map_start, map_end, grid_size, num_cargos, num_robots, alpha, 
            initial_positions, cargo_init_pos, cargo_goal_pos, id)

def preprocess_all(ids, size, n_sp, n_tars,n_robs):
    all_data = []

    for n_tar in n_tars:
        for n_rob in n_robs:
            for id in ids:
                result = preprocess(size, n_sp, n_tar, id,n_rob)
                (T, map_start, map_end, grid_size, num_cargos, num_robots, alpha,
                 initial_positions, cargo_init_pos, cargo_goal_pos, id) = result
                tar_count = len([item for item in cargo_goal_pos if item[0] is not None])

                all_data.append({
                    'T': T,
                    'map_start': map_start,
                    'map_end': map_end,
                    'grid_size': grid_size,
                    'num_cargos': num_cargos,
                    'num_robots': num_robots,
                    'alpha': alpha,
                    'initial_positions': initial_positions,
                    'cargo_init_pos': cargo_init_pos,
                    'cargo_goal_pos': cargo_goal_pos,
                    "id": id,
                    "num_targets":tar_count
            })    
    return all_data
